# Food Deserts in Washington DC
### *A comparison of food stability and resources between wards*

In [2]:
#Setting up environment by importing libraries and connecting to BigQuery

import os 
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/jacksonfuller/Downloads/food-inequity-and-disparities-a59dffd5fedf.json'
import pandas as pd

%load_ext google.cloud.bigquery
%matplotlib inline


### Demographic Characteristics by Ward



This query creates a table listing the demographic variables for each ward including income, race, and access to transportation

In [3]:
%%bigquery

#Pulls median household income data for each ward
WITH income AS (
    SELECT 
        NAMELSAD AS ward,
        DP03_0062E AS median_household_income
    FROM `food-inequity-and-disparities.dchealth.economic_characteristics` 
    ORDER BY ward
),

#Pulls racial breakdown of each ward with minority percentage defined as "Not white, not hispanic/latino"
race_percentage AS (
    SELECT
        NAMELSAD AS ward,
        ROUND(DP05_0077E / DP05_0070E *100,2) AS percentage_white_only
    FROM `food-inequity-and-disparities.dchealth.demographic_characteristics`
    ORDER BY ward
),

#Counts the number of metro stations in each ward
metro_count AS (
    
    #Cleans the metro station address data by matching the fortmatting of DC address table and filtering by only DC stations
    WITH metro_address AS (
        SELECT
            name,
            REGEXP_EXTRACT(address, r'[^,]*') AS dc_address
        FROM
            `food-inequity-and-disparities.dchealth.metro_stations`
        WHERE
            ENDS_WITH(address, 'DC')
    ),
    
    #Uses the DC address table to add ward information for each metro address 
    metro_ward AS (
        SELECT
            m.name,
            ad.ward
        FROM metro_address AS m
        LEFT JOIN `food-inequity-and-disparities.dchealth.address_list` AS ad
        ON m.dc_address = ad.fulladdress
    )
    
    #Counts the number of metro stations in each ward, excluding stations that aren't considered within a ward
    SELECT
        ward,
        COUNT(ward) AS metro_station_count
    FROM
        metro_ward
    WHERE
        ward IS NOT NULL
    GROUP BY ward
    ORDER BY ward
),

#Pulls the percentage of housheolds with at least 1 household vehicle available
vehicle_percentage AS (
    SELECT
        namelsad AS ward,
        ROUND(((DP04_0059E + DP04_0060E + DP04_0061E) / DP04_0057E *100), 2) AS percentage_households_with_vehicle_available 
    FROM `food-inequity-and-disparities.dchealth.housing_characteristics`
    GROUP BY 
        ward,
        percentage_households_with_vehicle_available
    ORDER BY ward
),

#Combines demographic variables into a single table
demographics AS (
    SELECT
        i.ward,
        i.median_household_income,
        r.percentage_white_only,
        m.metro_station_count,
        v.percentage_households_with_vehicle_available
    FROM income AS i
    JOIN race_percentage AS r ON i.ward=r.ward
    JOIN metro_count AS m ON i.ward=m.ward
    JOIN vehicle_percentage AS v ON i.ward=v.ward
    ORDER BY i.ward
)
SELECT *
FROM demographics

Downloading: 100%|██████████████████████████████| 8/8 [00:07<00:00,  1.06rows/s]


,ward,median_household_income,percentage_white_only,metro_station_count,percentage_households_with_vehicle_available
0,Ward 1,102882,45.92,2,54.25
1,Ward 2,111064,65.40,11,49.91
2,Ward 3,128670,70.86,4,77.52
3,Ward 4,94810,24.75,2,78.37
4,Ward 5,71782,23.26,4,69.07
5,Ward 6,114363,55.74,9,66.63
6,Ward 7,45318,2.66,3,61.63
7,Ward 8,35245,4.96,2,55.57



In order to get a preliminary idea of any wards that may be outliers, standard deviation analysis was used to find any wards that didn't fit within 80% confidence interval from the sample mean. This test assumed the ward data follows a normal distribution. While not statistically significant, this give us an idea of which wards to look more closely at in future analysis.

In [4]:
%%bigquery
WITH std_income AS (
    SELECT 
        d.ward, 
        CASE 
            WHEN median_household_income BETWEEN mean - 1.282 * stdev AND mean + 1.282 * stdev 
            THEN 'not outlier' 
            ELSE 'outlier' 
        END AS label_median_household_income
    FROM `food-inequity-and-disparities.dchealth.demographics` AS d
    CROSS JOIN (
        SELECT 
            AVG(d.median_household_income) AS mean, 
            STDDEV_SAMP(d.median_household_income) AS stdev
        FROM `food-inequity-and-disparities.dchealth.demographics` AS d) AS mean_sd
    ORDER BY ward
),
std_race_percentage AS (
    SELECT 
        d.ward, 
        CASE 
            WHEN percentage_white_only BETWEEN mean - 1.282 * stdev AND mean + 1.282 * stdev 
            THEN 'not outlier' 
            ELSE 'outlier' 
        END AS label_percentage_white_only
    FROM `food-inequity-and-disparities.dchealth.demographics` AS d
    CROSS JOIN (
        SELECT
            AVG(d.percentage_white_only) AS mean, 
            STDDEV_SAMP(d.percentage_white_only) AS stdev
        FROM `food-inequity-and-disparities.dchealth.demographics` AS d) AS mean_sd
    ORDER BY ward
),
std_metro_count AS (
    SELECT 
        d.ward, 
        CASE 
            WHEN metro_station_count BETWEEN mean - 1.282 * stdev AND mean + 1.282 * stdev 
            THEN 'not outlier' 
            ELSE 'outlier' 
        END AS label_metro_station_count
    FROM `food-inequity-and-disparities.dchealth.demographics` AS d
    CROSS JOIN (
        SELECT
            #Total number of metro stations in DC divided by the number of wards
            AVG(d.metro_station_count) AS mean, 
            STDDEV_SAMP(d.metro_station_count) AS stdev
        FROM `food-inequity-and-disparities.dchealth.demographics` AS d) AS mean_sd
    ORDER BY ward
),
std_vehicle_percentage AS (
    SELECT 
        d.ward, 
        CASE 
            WHEN percentage_households_with_vehicle_available BETWEEN mean - 1.282 * stdev AND mean + 1.282 * stdev 
            THEN 'not outlier' 
            ELSE 'outlier' 
        END AS label_percentage_households_with_vehicle_available
    FROM `food-inequity-and-disparities.dchealth.demographics` AS d
    CROSS JOIN (
        SELECT
            AVG(d.percentage_households_with_vehicle_available) AS mean, 
            STDDEV_SAMP(d.percentage_households_with_vehicle_available) AS stdev
        FROM `food-inequity-and-disparities.dchealth.demographics` AS d) AS mean_sd
    ORDER BY ward
),
std_demographics AS (
    SELECT
        i.ward,
        i.label_median_household_income,
        r.label_percentage_white_only,
        m.label_metro_station_count,
        v.label_percentage_households_with_vehicle_available
    FROM
        std_income AS i
    JOIN std_race_percentage AS r ON i.ward=r.ward
    JOIN std_metro_count AS m ON i.ward=m.ward
    JOIN std_vehicle_percentage AS v ON i.ward=v.ward
    ORDER BY i.ward
)
SELECT *
FROM std_demographics

Downloading: 100%|██████████████████████████████| 8/8 [00:07<00:00,  1.09rows/s]


,ward,label_median_household_income,label_percentage_white_only,label_metro_station_count,label_percentage_households_with_vehicle_available
0,Ward 1,not outlier,not outlier,not outlier,not outlier
1,Ward 2,not outlier,not outlier,outlier,outlier
2,Ward 3,not outlier,outlier,not outlier,not outlier
3,Ward 4,not outlier,not outlier,not outlier,outlier
4,Ward 5,not outlier,not outlier,not outlier,not outlier
5,Ward 6,not outlier,not outlier,not outlier,not outlier
6,Ward 7,not outlier,outlier,not outlier,not outlier
7,Ward 8,outlier,not outlier,not outlier,not outlier


###Food Resources Available by Ward

<br>This query creates a table delineating how many grocery stores and how many restaurants are in each ward

In [5]:
%%bigquery

#Uses DC Address table to find out which ward the fast food restaurants are located in
WITH fast_food AS (
    SELECT
        ff.name,
        ad.ward
    FROM `food-inequity-and-disparities.dchealth.fast_food_locations` AS ff
    LEFT JOIN `food-inequity-and-disparities.dchealth.address_list` AS ad 
    ON UPPER(ff.address) = UPPER(ad.fulladdress)
),
#Counts the number of fast food restaurants in each ward
fast_food_by_ward AS (
    SELECT 
        ward,
        COUNT(ward) AS fast_food_count
    FROM fast_food
    GROUP BY ward
    ORDER BY ward
),
#Counts the number of grocery stores in each ward
grocery_store_by_ward AS (
    SELECT
        ward,
        COUNT(ward) AS grocery_store_count
    FROM `food-inequity-and-disparities.dchealth.grocery_store_locations`
    GROUP BY ward
    ORDER BY ward
),
#Combines fast food restaurant count and grocery store count into a single table
food AS (
    SELECT
        ffw.ward,
        gsw.grocery_store_count,
        ffw.fast_food_count
    FROM fast_food_by_ward AS ffw
    LEFT JOIN grocery_store_by_ward AS gsw
    ON ffw.ward=gsw.ward
    ORDER BY ffw.ward
)
SELECT *
FROM food

Downloading: 100%|██████████████████████████████| 8/8 [00:07<00:00,  1.12rows/s]


,ward,grocery_store_count,fast_food_count
0,Ward 1,7,61
1,Ward 2,8,200
2,Ward 3,10,52
3,Ward 4,4,34
4,Ward 5,5,57
5,Ward 6,11,88
6,Ward 7,2,27
7,Ward 8,1,23



Again, the standard deviations of the variables, this time the food resource variables, away from the standard mean were tested to find any wards that didn't fit within 80% confidence interval from the sample mean.

In [6]:
%%bigquery
WITH std_grocery AS (
    SELECT 
        f.ward, 
        CASE 
            WHEN grocery_store_count BETWEEN mean - 1.282 * stdev AND mean + 1.282 * stdev 
            THEN 'not outlier' 
            ELSE 'outlier' 
        END AS label_grocery_store_count
    FROM `food-inequity-and-disparities.dchealth.food` AS f
    CROSS JOIN (
        SELECT
            AVG(f.grocery_store_count) AS mean, 
            STDDEV_SAMP(f.grocery_store_count) AS stdev
        FROM `food-inequity-and-disparities.dchealth.food` AS f) AS mean_sd
    ORDER BY ward
),
std_fast_food AS (
    SELECT 
        f.ward, 
        CASE 
            WHEN fast_food_count BETWEEN mean - 1.282 * stdev AND mean + 1.282 * stdev 
            THEN 'not outlier' 
            ELSE 'outlier' 
        END AS label_fast_food_count
    FROM `food-inequity-and-disparities.dchealth.food` AS f
    CROSS JOIN (
        SELECT
            AVG(f.fast_food_count) AS mean, 
            STDDEV_SAMP(f.fast_food_count) AS stdev
        FROM `food-inequity-and-disparities.dchealth.food` AS f) AS mean_sd
    ORDER BY ward
),
std_food AS (
    SELECT
        g.ward,
        g.label_grocery_store_count,
        ff.label_fast_food_count
    FROM std_grocery AS g
    JOIN std_fast_food AS ff ON g.ward=ff.ward
    ORDER BY g.ward
)
SELECT *
FROM std_food

Downloading: 100%|██████████████████████████████| 8/8 [00:07<00:00,  1.04rows/s]


,ward,label_grocery_store_count,label_fast_food_count
0,Ward 1,not outlier,not outlier
1,Ward 2,not outlier,outlier
2,Ward 3,not outlier,not outlier
3,Ward 4,not outlier,not outlier
4,Ward 5,not outlier,not outlier
5,Ward 6,outlier,not outlier
6,Ward 7,not outlier,not outlier
7,Ward 8,outlier,not outlier



### Relationships between Demographic and Food Variables in Each Ward


While admittedly a messy query, this query measures the relationships between the demographic and food resource variables for each ward, the Pearson correlation coefficient was calculated for each pair of variables and ranked from highest (strong correlation) to lowest (weak or no correlation)

In [7]:
%%bigquery
WITH dc_demographics_food AS (
    SELECT d.*, f.grocery_store_count, f.fast_food_count
    FROM `food-inequity-and-disparities.dchealth.demographics` AS d
    JOIN `food-inequity-and-disparities.dchealth.food` AS f ON d.ward=f.ward
),
income_grocery AS (
    SELECT
        (
            SELECT column_name
            FROM `food-inequity-and-disparities`.dchealth.INFORMATION_SCHEMA.COLUMNS
            WHERE table_name = 'demographics' AND column_name = 'median_household_income'
        ) AS demographic_variable,
        (
            SELECT column_name
            FROM `food-inequity-and-disparities`.dchealth.INFORMATION_SCHEMA.COLUMNS
            WHERE table_name = 'food' AND column_name = 'grocery_store_count'
        ) AS food_resource_variable,
        CORR(median_household_income,grocery_store_count) AS correlation
    FROM
        dc_demographics_food
),
income_fast_food AS (
    SELECT
        (
            SELECT column_name
            FROM `food-inequity-and-disparities`.dchealth.INFORMATION_SCHEMA.COLUMNS
            WHERE table_name = 'demographics' AND column_name = 'median_household_income'
        ) AS demographic_variable,
        (
            SELECT column_name
            FROM `food-inequity-and-disparities`.dchealth.INFORMATION_SCHEMA.COLUMNS
            WHERE table_name = 'food' AND column_name = 'fast_food_count'
        ) AS food_resource_variable,
        CORR(median_household_income,fast_food_count) AS correlation
    FROM
        dc_demographics_food
),
race_grocery AS (
    SELECT
        (
            SELECT column_name
            FROM `food-inequity-and-disparities`.dchealth.INFORMATION_SCHEMA.COLUMNS
            WHERE table_name = 'demographics' AND column_name = 'percentage_white_only'
        ) AS demographic_variable,
        (
            SELECT column_name
            FROM `food-inequity-and-disparities`.dchealth.INFORMATION_SCHEMA.COLUMNS
            WHERE table_name = 'food' AND column_name = 'grocery_store_count'
        ) AS food_resource_variable,
        CORR(percentage_white_only,grocery_store_count) AS correlation
    FROM
        dc_demographics_food
),
race_fast_food AS (
    SELECT
        (
            SELECT column_name
            FROM `food-inequity-and-disparities`.dchealth.INFORMATION_SCHEMA.COLUMNS
            WHERE table_name = 'demographics' AND column_name = 'percentage_white_only'
        ) AS demographic_variable,
        (
            SELECT column_name
            FROM `food-inequity-and-disparities`.dchealth.INFORMATION_SCHEMA.COLUMNS
            WHERE table_name = 'food' AND column_name = 'fast_food_count'
        ) AS food_resource_variable,
        CORR(percentage_white_only,fast_food_count) AS correlation
    FROM
        dc_demographics_food
),
metro_grocery AS (
    SELECT
        (
            SELECT column_name
            FROM `food-inequity-and-disparities`.dchealth.INFORMATION_SCHEMA.COLUMNS
            WHERE table_name = 'demographics' AND column_name = 'metro_station_count'
        ) AS demographic_variable,
        (
            SELECT column_name
            FROM `food-inequity-and-disparities`.dchealth.INFORMATION_SCHEMA.COLUMNS
            WHERE table_name = 'food' AND column_name = 'grocery_store_count'
        ) AS food_resource_variable,
        CORR(metro_station_count,grocery_store_count) AS correlation
    FROM
        dc_demographics_food
),
metro_fast_food AS (
    SELECT
        (
            SELECT column_name
            FROM `food-inequity-and-disparities`.dchealth.INFORMATION_SCHEMA.COLUMNS
            WHERE table_name = 'demographics' AND column_name = 'metro_station_count'
        ) AS demographic_variable,
        (
            SELECT column_name
            FROM `food-inequity-and-disparities`.dchealth.INFORMATION_SCHEMA.COLUMNS
            WHERE table_name = 'food' AND column_name = 'fast_food_count'
        ) AS food_resource_variable,
        CORR(metro_station_count,fast_food_count) AS correlation
    FROM
        dc_demographics_food
),
vehicle_grocery AS (
    SELECT
        (
            SELECT column_name
            FROM `food-inequity-and-disparities`.dchealth.INFORMATION_SCHEMA.COLUMNS
            WHERE table_name = 'demographics' AND column_name = 'percentage_households_with_vehicle_available'
        ) AS demographic_variable,
        (
            SELECT column_name
            FROM `food-inequity-and-disparities`.dchealth.INFORMATION_SCHEMA.COLUMNS
            WHERE table_name = 'food' AND column_name = 'grocery_store_count'
        ) AS food_resource_variable,
        CORR(percentage_households_with_vehicle_available,grocery_store_count) AS correlation
    FROM
        dc_demographics_food
),
vehicle_fast_food AS (
    SELECT
        (
            SELECT column_name
            FROM `food-inequity-and-disparities`.dchealth.INFORMATION_SCHEMA.COLUMNS
            WHERE table_name = 'demographics' AND column_name = 'percentage_households_with_vehicle_available'
        ) AS demographic_variable,
        (
            SELECT column_name
            FROM `food-inequity-and-disparities`.dchealth.INFORMATION_SCHEMA.COLUMNS
            WHERE table_name = 'food' AND column_name = 'fast_food_count'
        ) AS food_resource_variable,
        CORR(percentage_households_with_vehicle_available,fast_food_count) AS correlation
    FROM
        dc_demographics_food
),
correlations AS (
    #UNION ALL TABLES
    SELECT *
    FROM income_grocery
    UNION ALL
    SELECT *
    FROM income_fast_food
    UNION ALL
    SELECT *
    FROM race_grocery
    UNION ALL
    SELECT *
    FROM race_fast_food
    UNION ALL
    SELECT *
    FROM metro_grocery
    UNION ALL
    SELECT *
    FROM metro_fast_food
    UNION ALL
    SELECT *
    FROM vehicle_grocery
    UNION ALL
    SELECT *
    FROM vehicle_fast_food
)
SELECT *
FROM correlations
ORDER BY ABS(correlation) DESC

Downloading: 100%|██████████████████████████████| 8/8 [00:02<00:00,  3.39rows/s]


,demographic_variable,food_resource_variable,correlation
0,percentage_white_only,grocery_store_count,0.936653
1,median_household_income,grocery_store_count,0.922922
2,metro_station_count,fast_food_count,0.889361
3,percentage_white_only,fast_food_count,0.649954
4,metro_station_count,grocery_store_count,0.626113
5,median_household_income,fast_food_count,0.506009
6,percentage_households_with_vehicle_available,fast_food_count,-0.472812
7,percentage_households_with_vehicle_available,grocery_store_count,0.174554



### Conclusions


Though this analysis of food insecurity in Washington, DC we can see very strong correlations between race/income and grocery stores in Washington DC. This shows the systemic factors behind food insecurity in Washington, DC. There is also a moderate correlation between the number of metro stations in each ward and the number of grocery stores in a ward. These correlations reveal the inequal nature of resources in Washington, DC that lead to food inequity. In order to resolve this in the short term, the city of Washington, DC requires mandates for new grocery stores and fresh food locations in Wards 7 and 8. However, for long-term success, the city should consider addressing the underlying factors by introducing programs to increase access to postsecondary institutions, enforcement of labor laws to reduce exploitation of low-skilled workers, and providing greater availabilities for low-income residents to find housing throughout all of DC instead of in segregated neighborhoods.